# A. Euclidean Metric (with momentum) for linear and ReLU neural networks

As the study of the Euclidean GF for linear and ReLU neural networks has been done in [1] ([Associated code](https://github.com/sibyllema/Conservation_laws)), we only treat the Euclidean MF in this section.

[1]: Marcotte, Gribonval, Peyré. *Abide by the law and Follow the flow: Conservation laws for gradient flows. Neurips 23*.

## A. 1. The case of $q$-layer linear neural networks 

We consider the case of a $q$-layer linear neural network $g(\theta, x) = (U_1 \times  \cdots \times U_q)x$ where $\theta = (U_i)_i$ with $U_i$ the weight matrices.

We first treat the case of $\tau(t) \equiv \tau$. If  $\tau \neq 0$, as explained in Section 3.1.1, we use the change of variable $s = \exp(\tau t)$ in that case.

In [3]:
import numpy as np
def vector_fields_for_q_layer_LNN_momentum(list_dim, tau): #for example list_dim = [n1, n2, n3, n4]
    """
    Returns 2D+1 and the associated vector fields
    """
    D = int(np.array(list_dim)[:-1] @ np.array(list_dim)[1:].T)
 
    list_var = [var('x'+str(i+1)) for i in range(2*D + 1)]
    R = PolynomialRing(ZZ, list_var) 
    
    matrix_list = []
    for i in range(len(list_dim)-1):
        matrix = []
        for j in range(list_dim[i]):
            line = []
            for k in range(list_dim[i+1]):
                L = [0]*(2*D+1)
                L[j * list_dim[i+1] + k + sum([list_dim[l] * list_dim[l+1] for l in range(i)]) + 1] = 1
                line.extend([R.monomial(*L)])
            matrix.append(line)
        matrix_list.append(np.array(matrix))
  
    prod = np.linalg.multi_dot(matrix_list)
    assert prod.shape == (list_dim[0], list_dim[-1])
    vec = []
    for i in range(list_dim[0]):
        for j in range(list_dim[-1]):
            vec.append(prod[i][j].gradient())
               
    vec = np.array(vec).T[1:D+1]
    A = []
    for i in range(D+1):
        line = []
        for j in range(np.shape(vec)[1]):
            L = [0]*(2*D + 1)
            line.extend([R.monomial(*L)-1])
        A.append(line)
    B = np.concatenate((A, vec), axis=0)

    
    # build the extra vector field for momentum case
    extra = [0]*(2*D+1)
    monomials_list = [0]*(2*D+1)
    if tau == 0:
        extra[0] = R.monomial(*monomials_list)
    else:
        monomials_list[0] += 1
        extra[0] = tau * R.monomial(*monomials_list)
    for i in range(1, D+1):
        monomials_list = [0]*(2*D+1)
        monomials_list[i+ D] += 1
        extra[i] = R.monomial(*monomials_list)
        extra[i + D] = - tau* R.monomial(*monomials_list)
    extra = np.array(extra).reshape(2*D +1, 1)
    vec = np.concatenate((B, extra), axis = 1)
    
    return 2*D+1, vec

We now treat the case $\tau(t) := 3/t$ (Nesterov)


In [33]:
def vector_fields_for_q_layer_LNN_Nesterov(list_dim): #for example list_dim = [n1, n2, n3, n4]
    """
    Returns 2D+1 and the associated vector fields 
    """
    D = int(np.array(list_dim)[:-1] @ np.array(list_dim)[1:].T)
 
    list_var = [var('x'+str(i+1)) for i in range(2*D + 1)]
    R = PolynomialRing(ZZ, list_var)
    
    # to handle fractional coefficients
    F = R.fraction_field() 
    matrix_list = []
    for i in range(len(list_dim)-1):
        matrix = []
        for j in range(list_dim[i]):
            line = []
            for k in range(list_dim[i+1]):
                L = [0]*(2*D+1)
                L[j * list_dim[i+1] + k + sum([list_dim[l] * list_dim[l+1] for l in range(i)]) + 1] = 1
                line.extend([R.monomial(*L)])
            matrix.append(line)
        matrix_list.append(np.array(matrix))
  
    prod = np.linalg.multi_dot(matrix_list)
    assert prod.shape == (list_dim[0], list_dim[-1])
    vec = []
    for i in range(list_dim[0]):
        for j in range(list_dim[-1]):
            vec.append(prod[i][j].gradient())
               
    vec = np.array(vec).T[1:D+1]
    A = []
    for i in range(D+1):
        line = []
        for j in range(np.shape(vec)[1]):
            L = [0]*(2*D + 1)
            line.extend([R.monomial(*L)-1])
        A.append(line)
    B = np.concatenate((A, vec), axis=0)

    # build the extra vector field for momentum case
    extra = [0]*(2*D+1)
    monomials_list = [0]*(2*D+1)
    extra[0] = R.monomial(*monomials_list)
    for i in range(1, D+1):
        monomials_list = [0]*(2*D+1)
        monomials_list[i+ D] += 1
        extra[i] = R.monomial(*monomials_list)
        second_monomials_list = [0]*(2*D+1)
        second_monomials_list[0]+=1
        
        
        extra[i + D] = - 3/ list_var[0] * R.monomial(*monomials_list)
    extra = np.array(extra).reshape(2*D +1, 1)
    vec = np.concatenate((B, extra), axis = 1)
    
    return 2*D+1, vec

## A. 2. The case of $q$-layer ReLu neural networks 

We consider the case of a $q$-layer ReLU neural network with bias or not.

In [34]:
import itertools
from sage.parallel.decorate import parallel
import itertools

# Create function for operations inside first loop
@parallel
def compute_gradient_1(dims, matrix_list, bias_list, num_mat):
    vec = []
    to_multiply = [matrix_list[0].copy()[:, dims[0]]]
    for i in range(len(dims) - 1):
        to_multiply.append(np.array(matrix_list[i + 1].copy()[dims[i], dims[i+1]]))
    to_multiply.append(np.array(bias_list.copy()[num_mat - 1][dims[-1]]))
    prod = np.array(product(to_multiply)).reshape(-1)
    for l in range(len(prod)):
        vec.append(prod[l].gradient())
    return vec

# Create function for operations inside second loop (last layer)
@parallel
def compute_gradient_2(dims, matrix_list):
    vec = []
    to_multiply = [matrix_list[0].copy()[:, dims[0]]]
    for i in range(len(dims) - 1):
        to_multiply.append(np.array(matrix_list[i + 1].copy()[dims[i], dims[i+1]]))
    prod = np.array(product(to_multiply)).reshape(-1, 1) @ np.array(matrix_list[-1].copy()[dims[-1], :]).reshape(1, -1)
    for l in range(prod.shape[0]):
        for m in range(prod.shape[1]):
            vec.append(prod[l][m].gradient())
    return vec

def vector_fields_ReLU_q_layers_momentum(list_dim, tau, bias = False):
    D = np.array(list_dim)[:-1] @ np.array(list_dim)[1:].T
    dim_weights = D
    
    if bias:
        D += sum([list_dim[l] for l in range(1, len(list_dim)-1)])
        
    list_var = [var('x'+str(i+1)) for i in range(2*D +1)]
    R = PolynomialRing(ZZ, list_var) 
    
    # definition of the weight matrices
    matrix_list = []
    for i in range(len(list_dim)-1):
        matrix = []
        for j in range(list_dim[i]):
            line = []
            for k in range(list_dim[i+1]):
                L = [0]*(2*D+1)
                L[j * list_dim[i+1] + k + sum([list_dim[l] * list_dim[l+1] for l in range(i)])+1] = 1
                line.extend([R.monomial(*L)])
            matrix.append(line)
        matrix_list.append(np.array(matrix))
    
    # definition of the bias
    if bias:
        bias_list = []
        for i in range(1, len(list_dim)-1):
            line = []
            for j in range(list_dim[i]):
                L = [0]*(2*D+1)
                L[dim_weights + j + sum([list_dim[l] for l in range(1, i)])+1] += 1
                line.extend([R.monomial(*L)])
            bias_list.append(line)
    
        
    # Now use joblib to parallelize the for-loops
    vec = []
    if bias:
        for num_mat in range(1, len(list_dim) - 1):
            dims_list = itertools.product(*[list(range(list_dim[i])) for i in range(1, num_mat+1)])
            # use sage parallelization
            results = compute_gradient_1([(dims, matrix_list, bias_list, num_mat) for dims in dims_list])
            vec += [item for sublist in results for item in sublist[1]]  # Flatten results

    dims_list = itertools.product(*[list(range(list_dim[i])) for i in range(1, len(list_dim) - 1)])
    # use sage parallelization
    results = compute_gradient_2([(dims, matrix_list) for dims in dims_list])
    vec += [item for sublist in results for item in sublist[1]]  # Flatten results
    
    vec = np.array(vec).T[1:D+1]
    A = []
    for i in range(D+1):
        line = []
        for j in range(np.shape(vec)[1]):
            L = [0]*(2*D + 1)
            line.extend([R.monomial(*L)-1])
        A.append(line)
    B = np.concatenate((A, vec), axis=0)
    
    # build the extra vector field for momentum case
    extra = [0]*(2*D+1)
    monomials_list = [0]*(2*D+1)
    if tau ==0:
        extra[0] = R.monomial(*monomials_list) 
    else:
        # with the change of variables explained in 3.1.1 
        monomials_list[0] += 1
        extra[0] = tau * R.monomial(*monomials_list)
    
    # build all the others
    for i in range(1, D+1):
        monomials_list = [0]*(2*D+1)
        monomials_list[i+ D] += 1
        extra[i] = R.monomial(*monomials_list)
        extra[i + D] = - tau * R.monomial(*monomials_list)
    extra = np.array(extra).reshape(2*D+1, 1)
    vec = np.concatenate((B, extra), axis = 1)
    
    
    return 2*D+1, vec

In [4]:
def vector_fields_ReLU_q_layers_Nesterov(list_dim, bias = False):
    D = np.array(list_dim)[:-1] @ np.array(list_dim)[1:].T
    dim_weights = D
    
    if bias:
        D += sum([list_dim[l] for l in range(1, len(list_dim)-1)])
        
    list_var = [var('x'+str(i+1)) for i in range(2*D +1)]
    R = PolynomialRing(ZZ, list_var) 
    
    # to handle fractional coefficients
    F = R.fraction_field() 
    
    # definition of the weight matrices
    matrix_list = []
    for i in range(len(list_dim)-1):
        matrix = []
        for j in range(list_dim[i]):
            line = []
            for k in range(list_dim[i+1]):
                L = [0]*(2*D+1)
                L[j * list_dim[i+1] + k + sum([list_dim[l] * list_dim[l+1] for l in range(i)])+1] = 1
                line.extend([R.monomial(*L)])
            matrix.append(line)
        matrix_list.append(np.array(matrix))
    
    # definition of the bias
    if bias:
        bias_list = []
        for i in range(1, len(list_dim)-1):
            line = []
            for j in range(list_dim[i]):
                L = [0]*(2*D+1)
                L[dim_weights + j + sum([list_dim[l] for l in range(1, i)])+1] += 1
                line.extend([R.monomial(*L)])
            bias_list.append(line)
    
    # Now use joblib to parallelize the for-loops
    vec = []
    if bias:
        for num_mat in range(1, len(list_dim) - 1):
            dims_list = itertools.product(*[list(range(list_dim[i])) for i in range(1, num_mat+1)])
            # use sage parallelization
            results = compute_gradient_1([(dims, matrix_list, bias_list, num_mat) for dims in dims_list])
            vec += [item for sublist in results for item in sublist[1]]  # Flatten results

    dims_list = itertools.product(*[list(range(list_dim[i])) for i in range(1, len(list_dim) - 1)])
    # use sage parallelization
    results = compute_gradient_2([(dims, matrix_list) for dims in dims_list])
    vec += [item for sublist in results for item in sublist[1]]  # Flatten results
    
    vec = np.array(vec).T[1:D+1]
    A = []
    for i in range(D+1):
        line = []
        for j in range(np.shape(vec)[1]):
            L = [0]*(2*D + 1)
            line.extend([R.monomial(*L)-1])
        A.append(line)
    B = np.concatenate((A, vec), axis=0)
    
    # build the extra vector field for momentum case
    extra = [0]*(2*D+1)
    monomials_list = [0]*(2*D+1)
    extra[0] = R.monomial(*monomials_list) 
    
    # build all the others
    for i in range(1, D+1):
        monomials_list = [0]*(2*D+1)
        monomials_list[i+ D] += 1
        extra[i] = R.monomial(*monomials_list)
        extra[i + D] = - 3/ list_var[0] * R.monomial(*monomials_list)
    extra = np.array(extra).reshape(2*D+1, 1)
    vec = np.concatenate((B, extra), axis = 1)
    
    
    return 2*D+1, vec

# B. Example of non-Euclidean metric: the NMF

As the NMF is used for the matrix factorization, this is an example of a $2$-layer linear neural network. We write  $g(\theta, x) = U V x$. The non-negativity is contrained by multiplicative updates. For the gradient flow, the associated EDO writes: $\dot \theta = - \text{diag}(\theta) \nabla E_Z(\theta)$, and for the momentum flow with a constant $\tau$, the associated EDO writes:
$\ddot \theta + \tau \dot \theta = -  \text{diag}(\tau \theta + \dot \theta) \nabla E_Z(\theta)$.

## B. 1. The Gradient flow case.

In [5]:
def NMF(n1, n2, n3):
    size_V = n2*n3
    size_U = n1*n2
    D = size_U + size_V
    list_var = [var('x'+str(i+1)) for i in range(D)]
    R = PolynomialRing(ZZ, list_var) 
    
    matrix_hess = []
    for i in range(D):
        line = []
        for j in range(D):
            L = [0]*(D)
            if j == i:
                L[j] += 1
                line.extend([R.monomial(*L)])
            else:
                line.extend([R.monomial(*L)-1])
        matrix_hess.append(line)
    matrix_hess = np.array(matrix_hess)
    
    matrix_U = []
    for i in range(n1):
        line = []
        for j in range(n2):
            L = [0]*(D)
            L[i*n2 + j] += 1
            line.extend([R.monomial(*L)])
        matrix_U.append(line)
    U = np.array(matrix_U)
    matrix_V = []
    for i in range(n2):
        line = []
        for j in range(n3):
            L = [0]*D
            L[i*n3 + j + n1*n2] += 1
            line.extend([R.monomial(*L)])
        matrix_V.append(line)
    V = np.array(matrix_V)

    prod = U @ V

    vec = []
    for j in range(n3):
        for i in range(n1):
            vec_ = np.array(prod[i][j].gradient())
            mult = matrix_hess @ vec_.T
            vec.append(mult)
    vec = np.array(vec).T
    return D, vec

## B. 2. The momentum flow case.

In [14]:
def NMF_momentum(n1, n2, n3, tau):
    size_V = n2*n3
    size_U = n1*n2
    D = size_U + size_V
    list_var = [var('x'+str(i+1)) for i in range(2*D + 1)]
    R = PolynomialRing(ZZ, list_var) 
    
    matrix_hess = []
    for i in range(D):
        line = []
        for j in range(D):
            L = [0]*(2*D + 1)
            L_ = [0]*(2*D + 1)
            if j == i:
                L[j+1] += 1
                L_[D + j+1] += 1
                line.extend([tau*R.monomial(*L) + R.monomial(*L_)])
            else:
                line.extend([R.monomial(*L)-1])
        matrix_hess.append(line)
    matrix_hess = np.array(matrix_hess)
    
    matrix_U = []
    for i in range(n1):
        line = []
        for j in range(n2):
            L = [0]*(2*D+1)
            L[i*n2 + j+1] += 1
            line.extend([R.monomial(*L)])
        matrix_U.append(line)
    U = np.array(matrix_U)
    matrix_V = []
    for i in range(n2):
        line = []
        for j in range(n3):
            L = [0]*(2*D+1)
            L[i*n3 + j + n1*n2 +1] += 1
            line.extend([R.monomial(*L)])
        matrix_V.append(line)
    V = np.array(matrix_V)
    prod = U @ V

    vec = []
    for j in range(n3):
        for i in range(n1):
            vec_ = np.array(prod[i][j].gradient())
            mult = matrix_hess @ vec_.T[1:D+1]
            vec.append(mult)
    vec = np.array(vec).T
    A = []
    for i in range(D+1):
        line = []
        for j in range(np.shape(vec)[1]):
            L = [0]*(2*D + 1)
            line.extend([R.monomial(*L)-1])
        A.append(line)
    B = np.concatenate((A, vec), axis=0)
    
    # build the extra vector field for momentum case
    extra = [0]*(2*D+1)
    monomials_list = [0]*(2*D+1)
    if tau == 0:
        extra[0] = R.monomial(*monomials_list)
    else:
        monomials_list[0] += 1
        extra[0] = tau * R.monomial(*monomials_list)
    for i in range(1, D+1):
        monomials_list = [0]*(2*D+1)
        monomials_list[i+ D] += 1
        extra[i] = R.monomial(*monomials_list)
        extra[i + D] = - tau* R.monomial(*monomials_list)
    extra = np.array(extra).reshape(2*D +1, 1)
    vec = np.concatenate((B, extra), axis = 1)

    return 2*D+1, vec

# C. Another example for non-Euclidean metric: ICNN.

We only treat the $2$-layer case, and we write $g(\theta, x) = U \sigma(V + b)$, with $\sigma(t):=\max(t,0)$. We use mirror geometry on $U$ to constraint positivity  and Euclidean one on both $V$ and $b$.  For the gradient flow, the associated EDO writes: $\dot \theta = - \text{diag}(U, Id, Id) \nabla E_Z(\theta)$, and for the momentum flow with a constant $\tau$, the associated EDO writes:
$\ddot \theta + \tau \dot \theta = -  \text{diag}(\tau U+ \dot U, Id, Id) \nabla E_Z(\theta)$.

## C. 1. The Gradient flow case.

In [7]:
def ICNN(n1, n2, n3, bias = False):
    size_U = n1*n2
    size_V = n2*n3
    D = size_U + size_V
    if bias:
        D += n2
    list_var = [var('x'+str(i+1)) for i in range(D)]
    R = PolynomialRing(ZZ, list_var) 
    
    matrix_hess = []
    for i in range(D):
        line = []
        for j in range(D):
            if j == i:
                if i < size_U:
                    L = [0]*(D)
                    L[j] += 1
                    line.extend([R.monomial(*L)])
                else:
                    L = [0]*(D)
                    line.extend([R.monomial(*L)])    
            else:
                L = [0]*(D)
                line.extend([R.monomial(*L)-1])
        matrix_hess.append(line)
    matrix_hess = np.array(matrix_hess)
    
    matrix_U = []
    for i in range(n1):
        line = []
        for j in range(n2):
            L = [0]*(D)
            L[i*n2 + j] += 1
            line.extend([R.monomial(*L)])
        matrix_U.append(line)
    U = np.array(matrix_U)
    matrix_V = []
    for i in range(n2):
        line = []
        for j in range(n3):
            L = [0]*D
            L[i*n3 + j + n1*n2] += 1
            line.extend([R.monomial(*L)])
        matrix_V.append(line)
    V = np.array(matrix_V)
    
    if bias:
        bias_vector = []
        for i in range(n2):
            L = [0]*D
            L[n2*n3 + i + n1*n2] += 1
            bias_vector.append([R.monomial(*L)])
        bias_vector = np.array(bias_vector)
        
    vec = []
    for k in range(n2):
        prod_1 = U[ :, k].reshape(-1, 1) @ V[k,:].reshape(1, -1)
        if bias:
            prod_2 = U[ :, k].reshape(-1, 1) @ bias_vector[k].reshape(1, -1)
        for i in range(n1):
            if bias:
                vec_1 = np.array(prod_2[i][0].gradient())
                mult = matrix_hess @ vec_1.T
                vec.append(mult)
            for j in range(n3):
                vec_ = np.array(prod_1[i][j].gradient())
                mult = matrix_hess @ vec_.T
                vec.append(mult)

    vec = np.array(vec).T
    return D, vec

## C. 2. The Momentum flow case.

In [15]:
def ICNN_momentum(n1, n2, n3, tau, bias = False):
    size_U = n1*n2
    size_V = n2*n3
    D = size_U + size_V
    if bias:
        D += n2
    list_var = [var('x'+str(i+1)) for i in range(2*D+1)]
    R = PolynomialRing(ZZ, list_var) 
    
    matrix_hess = []
    for i in range(D):
        line = []
        for j in range(D):
            if j == i:
                if i < size_U:
                    L = [0]*(2*D+1)
                    L[j+1] += 1
                    line.extend([R.monomial(*L)])
                else:
                    L = [0]*(2*D+1)
                    line.extend([R.monomial(*L)])
            else:
                L = [0]*(2*D+1)
                line.extend([R.monomial(*L)-1])
        matrix_hess.append(line)
    matrix_hess = np.array(matrix_hess)
    
    matrix_U = []
    for i in range(n1):
        line = []
        for j in range(n2):
            L = [0]*(2*D+1)
            L[i*n2 + j+1] += 1
            line.extend([R.monomial(*L)])
        matrix_U.append(line)
    U = np.array(matrix_U)
    
    matrix_V = []
    for i in range(n2):
        line = []
        for j in range(n3):
            L = [0]*(2*D+1)
            L[i*n3 + j + n1*n2 +1] += 1
            line.extend([R.monomial(*L)])
        matrix_V.append(line)
    V = np.array(matrix_V)
    
    if bias:
        bias_vector = []
        for i in range(n2):
            L = [0]*(2*D+1)
            L[n2*n3 + i + n1*n2 +1] += 1
            bias_vector.append([R.monomial(*L)])
        bias_vector = np.array(bias_vector)
        
    vec = []
    for k in range(n2):
        prod_1 = U[ :, k].reshape(-1, 1) @ V[k,:].reshape(1, -1)
        if bias:
            prod_2 = U[ :, k].reshape(-1, 1) @ bias_vector[k].reshape(1, -1)
        for i in range(n1):
            if bias:
                vec_1 = np.array(prod_2[i][0].gradient())
                mult = matrix_hess @ vec_1.T[1:D+1]
                vec.append(mult)
            for j in range(n3):
                vec_2 = np.array(prod_1[i][j].gradient())
                mult = matrix_hess @ vec_2.T[1:D+1]
                vec.append(mult)

    vec = np.array(vec).T
    A = []
    for i in range(D+1):
        line = []
        for j in range(np.shape(vec)[1]):
            L = [0]*(2*D + 1)
            line.extend([R.monomial(*L)-1])
        A.append(line)
    B = np.concatenate((A, vec), axis=0)
    
    # build the extra vector field for momentum case
    extra = [0]*(2*D+1)
    monomials_list = [0]*(2*D+1)
    if tau == 0:
        extra[0] = R.monomial(*monomials_list)
    else:
        monomials_list[0] += 1
        extra[0] = tau * R.monomial(*monomials_list)
    for i in range(1, D+1):
        monomials_list = [0]*(2*D+1)
        monomials_list[i+ D] += 1
        extra[i] = R.monomial(*monomials_list)
        extra[i + D] = - tau* R.monomial(*monomials_list)
    extra = np.array(extra).reshape(2*D +1, 1)
    vec = np.concatenate((B, extra), axis = 1)
    
    return 2*D+1, vec